                               Why are students absent?

Introduction:
    New York City Public Schools are the largest public school system in the United States. As such, large datasets about all kinds of variables are created in order to facilitate the optimal education for all students. The data is publicly available and compiled by PASSNYC, a non-profit dedicated to broadening educational opportunities for New York City students. 
    In this project I am using data from this compiled dataset in order to answer the question "Why are students absent?". To answer this question, I am...

#explain why I picked these variables 

Data Cleaning:

In [36]:
#import all software needed
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [37]:
#read in data
df = pd.read_csv('2016 School Explorer.csv')

In [38]:
relevant_columns = ['Economic Need Index', 'Average ELA Proficiency', 'Average Math Proficiency', 'Trust %', 'Strong Family-Community Ties %', 'Student Attendance Rate', 'Percent of Students Chronically Absent']

#replace "N/A" to a null variable for relevant columns that contain "N/A"
df.replace('N/A', np.nan, inplace=True)

#check data for null variables in relevant columns
print(df[relevant_columns].isnull().any())

#check data for type
print(df[relevant_columns].dtypes)

Economic Need Index                       True
Average ELA Proficiency                   True
Average Math Proficiency                  True
Trust %                                   True
Strong Family-Community Ties %            True
Student Attendance Rate                   True
Percent of Students Chronically Absent    True
dtype: bool
Economic Need Index                       float64
Average ELA Proficiency                   float64
Average Math Proficiency                  float64
Trust %                                    object
Strong Family-Community Ties %             object
Student Attendance Rate                    object
Percent of Students Chronically Absent     object
dtype: object


In [39]:
columns_to_change = ['Trust %', 'Strong Family-Community Ties %', 'Student Attendance Rate', 'Percent of Students Chronically Absent']

#eliminate rows with null variables
df.dropna(subset=columns_to_change, inplace=True)

#convert columns to float values
df[columns_to_change] = df[columns_to_change].apply(lambda x: x.str.rstrip('%').astype('float') / 100.0)

#create table with all relevant variables
relevant_columns_table = df[relevant_columns]
relevant_columns_table

,Economic Need Index,Average ELA Proficiency,Average Math Proficiency,Trust %,Strong Family-Community Ties %,Student Attendance Rate,Percent of Students Chronically Absent
0,0.919,2.14,2.17,0.94,0.85,0.94,0.18
1,0.641,2.63,2.98,0.94,0.86,0.92,0.30
2,0.744,2.39,2.54,0.79,0.80,0.94,0.20
3,0.860,2.48,2.47,0.88,0.89,0.92,0.28
4,0.730,2.38,2.54,0.93,0.89,0.93,0.23
...,...,...,...,...,...,...,...
1267,0.832,2.48,2.60,0.94,0.83,0.95,0.13
1268,0.825,2.50,2.85,0.93,0.88,0.94,0.24
1269,0.724,2.77,3.09,0.94,0.86,0.95,0.12
1270,0.544,2.60,3.29,0.95,0.91,0.95,0.12


Create Summary Statistics Tables

In [40]:
#create seperate tables for each summary statistic
mean_values = relevant_columns_table[relevant_columns].mean()
median_values = relevant_columns_table[relevant_columns].median()
mode_values = relevant_columns_table[relevant_columns].mode()
variance_values = relevant_columns_table[relevant_columns].var()
std_values = relevant_columns_table[relevant_columns].std()
skew_values = relevant_columns_table[relevant_columns].skew()
iqr_values = relevant_columns_table[relevant_columns].quantile([0.25, 0.75])
kurtosis_values = relevant_columns_table[relevant_columns].kurtosis()
#possibly add range

#create dataframes for each summary statistic 
mean_df = pd.DataFrame(mean_values, columns=['Mean'])
median_df = pd.DataFrame(median_values, columns=['Median'])
mode_df = pd.DataFrame(mode_values, columns=['Mode'])
variance_df = pd.DataFrame(variance_values, columns=['Variance'])
std_df = pd.DataFrame(std_values, columns=['Standard Deviation'])
skew_df = pd.DataFrame(skew_values, columns=['Skewness'])
iqr_df = pd.DataFrame(iqr_values, columns=['Q1', 'Q3']).transpose()
kurtosis_df = pd.DataFrame(kurtosis_values, columns=['Kurtosis'])

#merge tables to show one clean output

summary_stats = pd.concat([mean_df, median_df, mode_df, variance_df, std_df, skew_df, iqr_df, kurtosis_df], axis=1)
print(summary_stats)

                                            Mean  Median  Mode  Variance  \
Economic Need Index                     0.672281   0.731   NaN  0.044504   
Average ELA Proficiency                 2.534215   2.450   NaN  0.132197   
Average Math Proficiency                2.668956   2.580   NaN  0.221342   
Trust %                                 0.904226   0.920   NaN  0.003749   
Strong Family-Community Ties %          0.830914   0.830   NaN  0.003942   
Student Attendance Rate                 0.927249   0.940   NaN  0.007640   
Percent of Students Chronically Absent  0.215750   0.200   NaN  0.019801   
0                                            NaN     NaN   NaN       NaN   
1                                            NaN     NaN   NaN       NaN   
Q1                                           NaN     NaN   NaN       NaN   
Q3                                           NaN     NaN   NaN       NaN   

                                        Standard Deviation  Skewness  0.25  \
Economic 

In [41]:
correlation_matrix = relevant_columns_table[['economic_need_index', 'average_ELA_proficiency', 'average_math_proficiency', 'trust_percentage', 'strong_family_community_ties_percentage', 'student_attendance_rate_percent']].corr()

KeyError: "None of [Index(['economic_need_index', 'average_ELA_proficiency',\n       'average_math_proficiency', 'trust_percentage',\n       'strong_family_community_ties_percentage',\n       'student_attendance_rate_percent'],\n      dtype='object')] are in the [columns]"